In [1]:
from selenium import webdriver
import pandas as pd
import os
import sys
from os import listdir
from os.path import isfile, join
import time
import datetime
from datetime import timedelta, date
import itertools
import random
import requests
from bs4 import BeautifulSoup
from random import choice
from fake_useragent import UserAgent

In [3]:
driver = webdriver.Chrome()